In [43]:
import pandas as pd
import numpy as np
from time import time
from sklearn.ensemble import RandomForestClassifier

In [46]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

raw_data = pd.read_csv("/Users/samanthawise/Documents/VersionControl/datasciencetoolbox/assignment_1/data/kddcup.data_10_percent", header=None, names = col_names)
raw_data['label'] = raw_data['label'].replace(['back.', 'buffer_overflow.', 'ftp_write.', 'guess_passwd.', 'imap.', 'ipsweep.', 'land.', 'loadmodule.', 'multihop.', 'neptune.', 'nmap.', 'perl.', 'phf.', 'pod.', 'portsweep.', 'rootkit.', 'satan.', 'smurf.', 'spy.', 'teardrop.', 'warezclient.', 'warezmaster.'], 'attack')

In [47]:
# convert categoricals to ordinal (in-place)

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler

le = LabelEncoder()
hot = OneHotEncoder(sparse=True, handle_unknown='ignore')
minmax = MinMaxScaler()

def categorical_converter(data):
    '''manually encode the three categorical columns into ordinals'''
    cols=['protocol_type','service','flag']
    for i in cols:
        data[i] = le.fit_transform(data[i])
    data = hot.fit_transform(data.loc[:,cols])
    
categorical_converter(raw_data)

# In order to avoid some features distances dominate others, we need to scale all of them.

features= raw_data.iloc[:,:raw_data.shape[1]-1]
labels= raw_data.iloc[:,raw_data.shape[1]-1:].values.ravel()


raw_data.iloc[:,:raw_data.shape[1]-1] = minmax.fit_transform(raw_data.iloc[:,:raw_data.shape[1]-1])

In [48]:
# splitting our training and testing dataset 

from sklearn.model_selection import train_test_split

df= pd.DataFrame(features)

X_train, X_test, y_train, y_test = train_test_split(df, labels, train_size=0.8, test_size=0.2)
print("X_train, y_train:", X_train.shape, y_train.shape)
print("X_test, y_test:", X_test.shape, y_test.shape)

X_train, y_train: (395216, 41) (395216,)
X_test, y_test: (98805, 41) (98805,)


In [49]:
# training model

clf = RandomForestClassifier(n_jobs=-1, random_state=3, n_estimators=102)#, max_features=0.8, min_samples_leaf=3, n_estimators=500, min_samples_split=3, random_state=10, verbose=1)

t0 = time()

trained_model= clf.fit(X_train, y_train)

tt = time()-t0

print("Classifier trained in {} seconds".format(round(tt,3)))
print("Score: ", trained_model.score(X_train, y_train))

Classifier trained in 9.344 seconds
Score:  1.0


In [50]:
# predicting

y_pred = clf.predict(X_test)
y_pred

array(['attack', 'attack', 'attack', ..., 'attack', 'attack', 'attack'],
      dtype=object)

In [51]:
from sklearn.metrics import confusion_matrix, zero_one_loss

results = confusion_matrix(y_test, y_pred)
error = zero_one_loss(y_test, y_pred)

print("Confusion matrix:\n", results)

Confusion matrix:
 [[79339    14]
 [    3 19449]]


In [53]:
print("Accuracy: ", 1- error)

Accuracy:  0.9998279439299631
